# 유치원 수에 영향을 미치는 요소 파악
- 종속변수 : 행정동별 유치원 수
- 독립변수
    - 유해업소 수 : 유흥주점, 숙박업소 등
    - 복지시설 수 : 도서관, 병원, 공원 등
    - 법규관련 : LPG충전소, 주유소, 어린이집 등
        >공동주택, 어린이놀이터, 의료시설, 유치원, 보육시설, 경로당은 각 50m이상 떨어져야 하고,  
        >주유소는 25미터 이상, 자동차용 천연가스 충전소는 30미터 이상 떨어진 곳에 배치.
    - 교통 관련 데이터 : 고가도로, 터널, 육교, 횡단보도 수
    - 교육비 지출 금액 
    - 아파트 매매 가격

In [1]:
import pandas as pd
import numpy as np

## 종속변수 전처리
- 행정동별 유치원 수

In [139]:
df_kdrd = pd.read_csv('../data/raw_data/가중치선정/서울 유치원 전체(위경도추가).csv', encoding = 'ansi')
df_kdrd.head()

,교육지원청명,자치구,유치원명,공립/사립,유형,주소,Unnamed: 6,Latitude,Longitude,운영시간,...,만3세유아수,만4세유아수,만5세유아수,혼합유아수,특수유아수,보직교사수,일반교사수,특수교사수,기간제교사수,총 교사수
0,남부교육지원청,금천구,서울정심초등학교병설유치원(금천구),공립,병설,서울특별시 금천구 독산로78다길 89,NaN,37.471532,126.908285,07시30분~19시30분,...,3,15,16,0,4,1,2,1,0,4
1,강동송파교육지원청,송파구,서울솔방울유치원(송파구),공립,단설,서울특별시 송파구 오금로24길 25,NaN,37.504783,127.117784,07시00분~20시00분,...,28,31,43,0,6,2,4,0,5,11
2,강동송파교육지원청,송파구,란키즈유치원(송파구),사립,사인,서울특별시 송파구 새말로8길 22-5,NaN,37.480770,127.126272,08시00분~19시00분,...,23,31,21,0,0,0,6,0,0,6
3,동부교육지원청,중랑구,서울신현초등학교병설유치원(중랑구),공립,병설,서울특별시 중랑구 봉화산로 188,NaN,37.605671,127.093637,07시30분~19시30분,...,12,14,20,0,4,0,3,1,2,6
4,동부교육지원청,중랑구,서울신묵초등학교병설유치원(중랑구),공립,병설,서울특별시 중랑구 동일로149길 46,NaN,37.607168,127.075550,07시30분~19시30분,...,13,11,9,0,0,0,3,0,0,3


## 독립변수 전처리

### 1. 유해업소 관련 전처리

#### 1_1. 행정동별 유흥주점 수

In [140]:
df_1_1 = pd.read_csv('../data/raw_data/가중치선정/서울시 유흥주점영업 인허가 정보.csv', encoding = 'ansi')
df_1_1.head()

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,공장판매직종업원수,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
0,3220000,3220000-102-2018-00006,2018-12-11,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3230000,3230000-102-2017-00004,2017-05-31,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3130000,3130000-102-2002-00004,20021029,NaN,3,폐업,2,폐업,20130620,NaN,...,NaN,NaN,NaN,NaN,NaN,N,82.4,NaN,NaN,NaN
3,3180000,3180000-102-2001-14135,2001-10-19,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3210000,3210000-102-1982-06377,1982-02-06,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
df_1_1 = df_1_1[['상세영업상태명','지번주소', '사업장명', '업태구분명', '좌표정보(X)', '좌표정보(Y)']]
df_1_1 = df_1_1[df_1_1['상세영업상태명'] == '영업']
df_1_1.head()

,상세영업상태명,지번주소,사업장명,업태구분명,좌표정보(X),좌표정보(Y)
0,영업,"서울특별시 강남구 역삼동 678-24 늘봄빌딩 2,3층",클라스,룸살롱,203330.125000,444424.745000
1,영업,서울특별시 송파구 가락동 98-7,골드1,룸살롱,210491.321884,443569.763243
3,영업,서울특별시 영등포구 신길동 449-22 신성빌딩,린스,룸살롱,193084.355458,445642.841120
4,영업,서울특별시 서초구 잠원동 12-21,만남,룸살롱,201601.185141,446044.757176
6,영업,서울특별시 동대문구 장안동 382-23,아지트7080,룸살롱,206020.863854,451931.503922


In [142]:
df_1_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1749 entries, 0 to 4927
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   상세영업상태명  1749 non-null   object 
 1   지번주소     1748 non-null   object 
 2   사업장명     1749 non-null   object 
 3   업태구분명    1749 non-null   object 
 4   좌표정보(X)  1740 non-null   float64
 5   좌표정보(Y)  1740 non-null   float64
dtypes: float64(2), object(4)
memory usage: 95.6+ KB


In [143]:
df_1_1 = df_1_1.dropna(subset = ['지번주소']).reset_index(drop = True)

In [144]:
addr = df_1_1['지번주소'].str.split()

In [145]:
dong_list = []

for add in addr:
    dong_list.append(add[2])

# dong_list

In [146]:
gu_list = []

for add in addr:
    gu_list.append(add[1])

# gu_list

In [147]:
df_1_1['법정동'] = dong_list
df_1_1['자치구'] = gu_list
# df_1_1

In [148]:
df_1_1_new = df_1_1.groupby(['법정동']).count()[['자치구']].reset_index().rename(columns = {'자치구':'유흥주점 수'})

In [149]:
df_1_1_new

,법정동,유흥주점 수
0,가락동,41
1,가산동,13
2,가양동,2
3,갈월동,1
4,갈현동,13
...,...,...
182,회현동1가,2
183,효제동,15
184,휘경동,3
185,흑석동,1


In [150]:
df_1_1_new['법정동'].unique()
# ~~동 n가 ... -> 어떻게 처리해야하나...

array(['가락동', '가산동', '가양동', '갈월동', '갈현동', '개봉동', '견지동', '고척동', '공덕동',
       '공평동', '공항동', '관수동', '관철동', '관훈동', '광희동1가', '교남동', '교북동', '구의동',
       '군자동', '길동', '길음동', '낙원동', '남가좌동', '남대문로4가', '남대문로5가', '남산동2가',
       '남영동', '남창동', '남현동', '내수동', '노고산동', '노량진동', '녹번동', '논현동', '다동',
       '답십리동', '당산동1가', '당산동2가', '당산동5가', '당산동6가', '당주동', '대림동', '대방동',
       '대조동', '대치동', '대현동', '도렴동', '도선동', '도화동', '독산동', '돈의동', '동교동',
       '동선동1가', '동선동4가', '동소문동5가', '동소문동6가', '동자동', '마곡동', '마장동', '면목동',
       '목동', '묘동', '무교동', '무학동', '묵동', '문래동1가', '미아동', '반포동', '방배동',
       '방산동', '방이동', '방화동', '번동', '보문동4가', '봉천동', '북창동', '불광동', '사당동',
       '삼성동', '상계동', '상도동', '상봉동', '서교동', '서소문동', '서초동', '석촌동', '성내동',
       '송정동', '수송동', '수유동', '수표동', '숭인동', '시흥동', '신길동', '신당동', '신대방동',
       '신림동', '신문로1가', '신사동', '신설동', '신정동', '암사동', '양평동1가', '양평동4가',
       '여의도동', '역삼동', '연지동', '연희동', '염창동', '영등포동1가', '영등포동3가', '영등포동4가',
       '영등포동5가', '영등포동8가', '영천동', '오류동', '오장동', '와룡동', '용답동', '용두동',
       '

In [151]:
df_1_1.drop(columns='상세영업상태명', inplace=True)

In [152]:
df_1_1

,지번주소,사업장명,업태구분명,좌표정보(X),좌표정보(Y),법정동,자치구
0,"서울특별시 강남구 역삼동 678-24 늘봄빌딩 2,3층",클라스,룸살롱,203330.125000,444424.745000,역삼동,강남구
1,서울특별시 송파구 가락동 98-7,골드1,룸살롱,210491.321884,443569.763243,가락동,송파구
2,서울특별시 영등포구 신길동 449-22 신성빌딩,린스,룸살롱,193084.355458,445642.841120,신길동,영등포구
3,서울특별시 서초구 잠원동 12-21,만남,룸살롱,201601.185141,446044.757176,잠원동,서초구
4,서울특별시 동대문구 장안동 382-23,아지트7080,룸살롱,206020.863854,451931.503922,장안동,동대문구
...,...,...,...,...,...,...,...
1743,서울특별시 금천구 독산동 293-5,국빈관관광나이트크럽,고고(디스코)클럽,190771.595590,440515.669035,독산동,금천구
1744,서울특별시 영등포구 영등포동3가 17-9 스카이모텔,샤넬,기타,191722.228678,446262.797865,영등포동3가,영등포구
1745,"서울특별시 서초구 잠원동 21-3 ,4호 (지하 1층)",더 레이스,카바레,201630.614056,445872.941217,잠원동,서초구
1746,"서울특별시 영등포구 여의도동 44-32 에리트빌팅 지하101,102,103호",플레이,룸살롱,193745.643373,446380.254633,여의도동,영등포구


In [153]:
# df_1_1.to_csv('1_1_유흥주점(좌표포함).csv', index = False)

In [154]:
# df_1_1_new.to_csv('1_1_유흥주점수.csv', index = False)

#### 1_2. 행정동별 단란주점 수

In [155]:
df_1_2 = pd.read_csv('../data/raw_data/가중치선정/서울시 단란주점영업 인허가 정보.csv', encoding = 'ansi')
df_1_2.head()

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,공장판매직종업원수,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
0,3200000,3200000-103-2023-00001,2023-03-02,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3110000,3110000-103-1996-04503,1996-04-15,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3160000,3160000-103-1995-06145,1995-10-23,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3060000,3060000-103-1995-06838,1995-08-16,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3120000,3120000-103-2003-00001,2003-10-01,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [156]:
df_1_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11583 entries, 0 to 11582
Data columns (total 44 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   개방자치단체코드   11583 non-null  int64  
 1   관리번호       11583 non-null  object 
 2   인허가일자      11583 non-null  object 
 3   인허가취소일자    0 non-null      float64
 4   영업상태코드     11583 non-null  int64  
 5   영업상태명      11583 non-null  object 
 6   상세영업상태코드   11583 non-null  int64  
 7   상세영업상태명    11583 non-null  object 
 8   폐업일자       9583 non-null   object 
 9   휴업시작일자     0 non-null      float64
 10  휴업종료일자     0 non-null      float64
 11  재개업일자      0 non-null      float64
 12  전화번호       10482 non-null  object 
 13  소재지면적      10495 non-null  object 
 14  소재지우편번호    11578 non-null  object 
 15  지번주소       11578 non-null  object 
 16  도로명주소      3376 non-null   object 
 17  도로명우편번호    3268 non-null   float64
 18  사업장명       11583 non-null  object 
 19  최종수정일자     11583 non-null  object 
 20  데이터갱신구

In [157]:
df_1_2 = df_1_2[df_1_2['상세영업상태명'] == '영업']
df_1_2.info()
# 지번주소가 3개의 Null값 존재

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 11582
Data columns (total 44 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   개방자치단체코드   2000 non-null   int64  
 1   관리번호       2000 non-null   object 
 2   인허가일자      2000 non-null   object 
 3   인허가취소일자    0 non-null      float64
 4   영업상태코드     2000 non-null   int64  
 5   영업상태명      2000 non-null   object 
 6   상세영업상태코드   2000 non-null   int64  
 7   상세영업상태명    2000 non-null   object 
 8   폐업일자       0 non-null      object 
 9   휴업시작일자     0 non-null      float64
 10  휴업종료일자     0 non-null      float64
 11  재개업일자      0 non-null      float64
 12  전화번호       1546 non-null   object 
 13  소재지면적      1943 non-null   object 
 14  소재지우편번호    1997 non-null   object 
 15  지번주소       1997 non-null   object 
 16  도로명주소      1972 non-null   object 
 17  도로명우편번호    1945 non-null   float64
 18  사업장명       2000 non-null   object 
 19  최종수정일자     2000 non-null   object 
 20  데이터갱신구분

In [158]:
def addr_maker(df, addr_lst):
    dong_list = []
    for add in addr_lst:
        dong_list.append(add[2])

    gu_list = []
    for add in addr_lst:
        gu_list.append(add[1])


    df['자치구'] = gu_list
    df['법정동'] = dong_list

def pre_DF(df):
    df = df[df['상세영업상태명'].str.contains('영업')]
    df = df[['지번주소', '사업장명', '업태구분명', '좌표정보(X)', '좌표정보(Y)']]
    df = df.dropna(subset = ['지번주소']).reset_index(drop = True)

    addr_maker(df, df['지번주소'].str.split())

    return df

In [159]:
df_1_2 = pre_DF(df_1_2)
df_1_2

,지번주소,사업장명,업태구분명,좌표정보(X),좌표정보(Y),자치구,법정동
0,서울특별시 관악구 신림동 530-31 상진,판도라,단란주점,192332.463248,442308.045654,관악구,신림동
1,서울특별시 은평구 구산동 321-98 지하1층,왓따7080,단란주점,191862.871845,456988.192685,은평구,구산동
2,서울특별시 구로구 구로동 153-3,서담주점,단란주점,190601.661796,443226.994309,구로구,구로동
3,서울특별시 중랑구 묵동 245-1,유토피아,단란주점,206813.057180,456188.141055,중랑구,묵동
4,서울특별시 서대문구 홍제동 173-2,테크노,단란주점,195024.356116,453992.743836,서대문구,홍제동
...,...,...,...,...,...,...,...
1992,서울특별시 금천구 독산동 296-16 지하1층,백상,단란주점,190804.003574,441018.620016,금천구,독산동
1993,서울특별시 금천구 독산동 159-1 지하1층,대형노래밤,단란주점,190999.733581,441169.149711,금천구,독산동
1994,서울특별시 금천구 독산동 1037-0 지하1층 (독산역길 3),스타,단란주점,191190.375303,440465.765059,금천구,독산동
1995,서울특별시 금천구 독산동 159-20 지하1층,별 노래바,단란주점,190963.034476,441110.169979,금천구,독산동


In [160]:
df_1_2_new = df_1_2.groupby(['법정동']).count()['자치구'].reset_index().rename(columns = {'자치구':'단란주점수'})
df_1_2_new

,법정동,단란주점수
0,가락동,32
1,가리봉동,7
2,가산동,5
3,가양동,6
4,갈월동,1
...,...,...
232,회현동1가,1
233,회현동2가,1
234,효제동,1
235,흑석동,3


In [161]:
# df_1_2.to_csv('1_2_단란주점(좌표포함).csv', index = False)

In [162]:
# df_1_2_new.to_csv('1_2_단란주점수.csv', index = False)

#### 1_3. 행정동별 복합유통게임제공업 수

In [163]:
df_1_3 = pd.read_csv('../data/raw_data/가중치선정/서울시 복합유통게임제공업 인허가 정보.csv', encoding = 'ansi')
df_1_3

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,음향시설여부,편의시설여부,소방시설여부,총게임기수,기존게임업외업종명,제공게임물명,공연장형태구분명,품목명,최초등록시점,지역구분명
0,3210000,CDFF2241032023000002,2023-04-14,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3060000,CDFF2241032012000002,2012-08-31,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3210000,CDFF2241032023000001,2023-03-10,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3070000,CDFF2241032023000001,2023-05-26,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3200000,CDFF2241032015000001,2015-04-03,NaN,3,폐업,03,폐업,2023-04-26,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4523,3240000,CDFF2241032013000002,20131024,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,전체이용가,NaN,NaN,NaN,NaN
4524,3240000,CDFF2241032011000002,20110921,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,195.0,NaN,NaN,NaN,NaN,NaN,NaN
4525,3240000,CDFF2241032017000003,20171026,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,전체이용가,NaN,NaN,NaN,NaN
4526,3240000,CDFF2241032017000001,20170126,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [164]:
df_1_3[df_1_3['상세영업상태명'] == '영업중'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1374 entries, 0 to 4527
Data columns (total 56 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   개방자치단체코드   1374 non-null   int64  
 1   관리번호       1374 non-null   object 
 2   인허가일자      1374 non-null   object 
 3   인허가취소일자    0 non-null      float64
 4   영업상태코드     1374 non-null   int64  
 5   영업상태명      1374 non-null   object 
 6   상세영업상태코드   1374 non-null   object 
 7   상세영업상태명    1374 non-null   object 
 8   폐업일자       0 non-null      object 
 9   휴업시작일자     0 non-null      float64
 10  휴업종료일자     0 non-null      float64
 11  재개업일자      0 non-null      float64
 12  전화번호       761 non-null    object 
 13  소재지면적      0 non-null      float64
 14  소재지우편번호    1170 non-null   object 
 15  지번주소       1374 non-null   object 
 16  도로명주소      1148 non-null   object 
 17  도로명우편번호    197 non-null    float64
 18  사업장명       1374 non-null   object 
 19  최종수정일자     1374 non-null   object 
 20  데이터갱신구분 

In [165]:
df_1_3['사업장명'].unique()

array(['나무 PC', '불독 PC방', '포탈 피시방 강남역점(Portal PC방 강남역점)', ...,
       '스타덤PC방(둔촌역본점)', '제논 피씨 플렉스(ZENON PC PLEX)', 'The Ten PC Cafe'],
      dtype=object)

- 일반적인 PC방이 대부분 : 일반게임제공업이 성인 게임방에 가까움
- 복합유통게임제공업은 사용하지 않는 것으로 판단

#### 1_4. 행정동별 비디오물감상실업(DVD방만 해당) 수

In [166]:
df_1_4 = pd.read_csv('../data/raw_data/가중치선정/서울시 비디오물감상실업 인허가 정보.csv', encoding = 'ansi')
df_1_4

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,음향시설여부,편의시설여부,소방시설여부,총게임기수,기존게임업외업종명,제공게임물명,공연장형태구분명,품목명,최초등록시점,지역구분명
0,3220000,CDFF1242012010000002,2010-09-14,NaN,3,폐업,03,폐업,2023-03-13,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3170000,CDFF1242012010000001,2010-07-28,NaN,3,폐업,03,폐업,2012-10-08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3240000,CDFF1242011999000002,1999-07-12,NaN,4,취소/말소/만료/정지/중지,35,직권말소,2023-02-27,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3000000,CDFF1242012009000002,20091029,NaN,4,취소/말소/만료/정지/중지,35,직권말소,20220412,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3000000,CDFF1242012007000002,20071211,NaN,4,취소/말소/만료/정지/중지,35,직권말소,20220412,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335,3220000,CDFF1242012022000001,20220630,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1336,3210000,CDFF1242011997000007,19971125,NaN,4,취소/말소/만료/정지/중지,35,직권말소,20220706,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1337,3140000,CDFF1242012001000005,20010724,NaN,4,취소/말소/만료/정지/중지,35,직권말소,20220711,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1338,3190000,CDFF1242011999000006,19990730,NaN,3,폐업,03,폐업,20220831,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [167]:
df_1_4[df_1_4['상세영업상태명'] == '영업중']

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,음향시설여부,편의시설여부,소방시설여부,총게임기수,기존게임업외업종명,제공게임물명,공연장형태구분명,품목명,최초등록시점,지역구분명
15,3170000,CDFF1242012023000001,2023-07-29,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,3000000,CDFF1242012004000002,20040906,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,3010000,CDFF1242012021000001,20211220,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
243,3030000,CDFF1242012002000001,20020823,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,근린상업지역
308,3040000,CDFF1242012007000001,20070115,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1331,3240000,CDFF1242011996000012,19960909,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1332,3240000,CDFF1242012000000001,20001223,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1333,3240000,CDFF1242012001000003,20010313,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1334,3240000,CDFF1242012002000001,20020731,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [168]:
def pre_DF_1(df):
    df = df[df['영업상태명'].str.contains('영업')]
    df = df[['지번주소', '사업장명', '업태구분명', '좌표정보(X)', '좌표정보(Y)']]
    df = df.dropna(subset = ['지번주소']).reset_index(drop = True)

    addr_maker(df, df['지번주소'].str.split())

    return df

df_1_4 = pre_DF_1(df_1_4)
df_1_4

,지번주소,사업장명,업태구분명,좌표정보(X),좌표정보(Y),자치구,법정동
0,서울특별시 금천구 독산동 1156 금천롯데캐슬골드파크4차,ㅇㄴㄹ,NaN,190799.832999,439885.941303,금천구,독산동
1,서울특별시 종로구 관철동 14-3번지 5층,럭셔리 수 DVD관,NaN,198744.767994,451930.939476,종로구,관철동
2,서울특별시 중구 을지로6가 18-96 4층,다락방 OTT,NaN,200640.845643,451721.924345,중구,을지로6가
3,서울특별시 성동구 하왕십리동 966-12번지,아트비디오감상실업,NaN,202827.036551,451146.886883,성동구,하왕십리동
4,서울특별시 광진구 구의동 546-11번지 지너스타워 지하 106호,엠엔지 DVD 영화관,NaN,208189.434497,448129.166313,광진구,구의동
...,...,...,...,...,...,...,...
106,서울특별시 강동구 성내동 443-33번지,디프,NaN,211716.731305,446924.613700,강동구,성내동
107,서울특별시 강동구 천호동 454-65번지 5층,DVD시네마,NaN,211146.068506,448479.444644,강동구,천호동
108,서울특별시 강동구 천호동 456번지 신라빌딩B1,현대DVD방,NaN,210825.177288,448573.814955,강동구,천호동
109,서울특별시 강동구 길동 447-1번지,씨엔에스디비디방,NaN,211986.914896,448201.841636,강동구,길동


In [169]:
df_1_4[df_1_4['사업장명'] == '(주)여행코리아']

,지번주소,사업장명,업태구분명,좌표정보(X),좌표정보(Y),자치구,법정동
9,서울특별시 광진구 구의동 246-3번지 민도빌딩 301호 246동 3호,(주)여행코리아,NaN,207564.476227,448369.568917,광진구,구의동


In [170]:
df_1_4['사업장명'].unique()
# DVD방이 아닌 목록 확인
notDVD = ['ㅇㄴㄹ', '악쓰는하마', '아트비디오감상실업', '(주)신우주항공', '(주)여행코리아', '서울시각장애인복지관송파점자도서관']

# 애매한 것들 ex) 21세기, 구구구, '마루', '키노', 'OK', 디프 
# 대부분 지도에 안나오고, 노래방(음흉한..)이 있는것으로 보임

In [171]:
df_1_4 = df_1_4[~df_1_4['사업장명'].isin(notDVD)]
df_1_4

,지번주소,사업장명,업태구분명,좌표정보(X),좌표정보(Y),자치구,법정동
1,서울특별시 종로구 관철동 14-3번지 5층,럭셔리 수 DVD관,NaN,198744.767994,451930.939476,종로구,관철동
2,서울특별시 중구 을지로6가 18-96 4층,다락방 OTT,NaN,200640.845643,451721.924345,중구,을지로6가
4,서울특별시 광진구 구의동 546-11번지 지너스타워 지하 106호,엠엔지 DVD 영화관,NaN,208189.434497,448129.166313,광진구,구의동
5,서울특별시 광진구 화양동 8-24번지 (2층),엔디비디,NaN,206122.114682,448756.377807,광진구,화양동
7,서울특별시 광진구 화양동 7-42번지,허리우드비디오감상실,NaN,206071.426953,448734.249359,광진구,화양동
...,...,...,...,...,...,...,...
106,서울특별시 강동구 성내동 443-33번지,디프,NaN,211716.731305,446924.613700,강동구,성내동
107,서울특별시 강동구 천호동 454-65번지 5층,DVD시네마,NaN,211146.068506,448479.444644,강동구,천호동
108,서울특별시 강동구 천호동 456번지 신라빌딩B1,현대DVD방,NaN,210825.177288,448573.814955,강동구,천호동
109,서울특별시 강동구 길동 447-1번지,씨엔에스디비디방,NaN,211986.914896,448201.841636,강동구,길동


In [172]:
df_1_4_new = df_1_4.groupby(['법정동']).count()[['자치구']].reset_index().rename(columns = {'자치구' : 'DVD방수'})
df_1_4_new

,법정동,DVD방수
0,가락동,1
1,가리봉동,1
2,갈현동,1
3,공릉동,1
4,관철동,1
5,구로동,1
6,구의동,1
7,길동,1
8,노량진동,3
9,동선동1가,1


In [173]:
# df_1_4.to_csv('1_3_DVD방(비디오물감상실업)(좌표포함).csv', index = False)

In [174]:
# df_1_4_new.to_csv('1_3_DVD방(비디오물감상실업)수.csv', index = False)

#### 1_5 행정동별 일반게임제공업 수

In [175]:
df_1_5 = pd.read_csv('../data/raw_data/가중치선정/서울시 일반게임제공업 인허가 정보.csv', encoding = 'ansi')
df_1_5

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,음향시설여부,편의시설여부,소방시설여부,총게임기수,기존게임업외업종명,제공게임물명,공연장형태구분명,품목명,최초등록시점,지역구분명
0,3050000,CDFF2241132022000002,2022-02-22,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3050000,CDFF2241132015000002,2015-03-23,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3110000,CDFF2241132022000003,2022-03-17,NaN,3,폐업,03,폐업,2023-03-07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3050000,CDFF2241132023000003,2023-03-30,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3160000,CDFF2241132023000001,2023-03-09,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10107,3200000,CDFF2241132022000005,2022-09-23,NaN,3,폐업,03,폐업,2023-06-07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10108,3150000,CDFF2241132022000004,2022-08-12,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10109,3160000,CDFF2241132014000001,2014-08-28,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10110,3110000,CDFF2241132022000006,2022-07-27,NaN,3,폐업,03,폐업,2023-11-01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [176]:
df_1_5[df_1_5['상세영업상태명'] == '영업중'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 302 entries, 0 to 10109
Data columns (total 56 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   개방자치단체코드   302 non-null    int64  
 1   관리번호       302 non-null    object 
 2   인허가일자      302 non-null    object 
 3   인허가취소일자    0 non-null      object 
 4   영업상태코드     302 non-null    int64  
 5   영업상태명      302 non-null    object 
 6   상세영업상태코드   302 non-null    object 
 7   상세영업상태명    302 non-null    object 
 8   폐업일자       0 non-null      object 
 9   휴업시작일자     0 non-null      float64
 10  휴업종료일자     0 non-null      float64
 11  재개업일자      0 non-null      float64
 12  전화번호       41 non-null     object 
 13  소재지면적      0 non-null      float64
 14  소재지우편번호    112 non-null    float64
 15  지번주소       302 non-null    object 
 16  도로명주소      289 non-null    object 
 17  도로명우편번호    223 non-null    float64
 18  사업장명       302 non-null    object 
 19  최종수정일자     302 non-null    object 
 20  데이터갱신구분 

In [177]:
df_1_5['사업장명']

0             킹게임랜드
1            환상게임랜드
2             대왕게임장
3            ET게임랜드
4             로얄바카라
            ...    
10107          신왕PC
10108       신현대게임랜드
10109        씨티게임랜드
10110    연신내 불새게임랜드
10111        불새게임랜드
Name: 사업장명, Length: 10112, dtype: object

In [178]:
df_1_5.dropna(subset='상세영업상태명', inplace=True)

In [179]:
df_1_5 = pre_DF(df_1_5)
df_1_5

,지번주소,사업장명,업태구분명,좌표정보(X),좌표정보(Y),자치구,법정동
0,서울특별시 동대문구 청량리동 770,킹게임랜드,NaN,203814.474073,453094.880884,동대문구,청량리동
1,서울특별시 동대문구 청량리동 291,환상게임랜드,NaN,203919.238239,453245.365781,동대문구,청량리동
2,서울특별시 동대문구 청량리동 249,ET게임랜드,NaN,204007.767059,453209.916468,동대문구,청량리동
3,서울특별시 구로구 가리봉동 135-9 1층,로얄바카라,NaN,190270.260953,441970.427503,구로구,가리봉동
4,서울특별시 도봉구 창동 650-47,제일게임랜드,NaN,202905.439654,460427.769520,도봉구,창동
...,...,...,...,...,...,...,...
304,서울특별시 중랑구 면목동 634-5 삼원빌딩,준게임랜드,NaN,207683.608994,453071.827544,중랑구,면목동
305,서울특별시 영등포구 영등포동5가 18-1,킹게임랜드,NaN,191691.167906,446411.007064,영등포구,영등포동5가
306,서울특별시 은평구 불광동 281-172 3층,삼성게임랜드,NaN,193802.267466,456419.663955,은평구,불광동
307,서울특별시 강서구 화곡동 927-1 웰라이빌아파트,신현대게임랜드,NaN,186029.723926,447381.645907,강서구,화곡동


In [180]:
df_1_5['사업장명'].unique()

array(['킹게임랜드', '환상게임랜드', 'ET게임랜드', '로얄바카라', '제일게임랜드', '둘리게임랜드', '성심 게임장',
       '낙원게임장', '스타게임랜드', '황금게임랜드', '땅콩 게임장', '세븐게임랜드', '서울게임장', '중앙게임장',
       '더영등포게임랜드', '미아리게임랜드', '스카이게임랜드', '신길플러스게임장', '화룡게임랜드', '세븐게임장',
       '오류게임랜드', '로얄 연신내 게임랜드', '에이스게임랜드', '상봉게임랜드', '뉴스타게임랜드', '신림 게임랜드',
       '아레나 게임랜드', '하이게임랜드', '로얄게임랜드', '포커성 게임랜드', '홍제게임랜드', '장안게임랜드',
       '럭키게임장', '난곡성인게임장', '강서게임랜드', '할리스 연신내 게임랜드', '삼성게임랜드 메트로',
       '화신성게임랜드', '명성 게임장', '행운게임랜드', '골든게임랜드', '대빵오락실', '불광게임랜드',
       '대박게임장', '대박 게임장', '수유역 게임랜드', '우주 게임장', '팡팡게임랜드', '상봉게임장',
       '낙원게임랜드', '파고다 게임랜드', '왕 게임랜드', '대박게임랜드', '스타게임장', '종합게임랜드',
       '튤립게임랜드', '삼정게임랜드', '화이트게임랜드', '매직게임랜드', '무진게임랜드', '까치게임랜드',
       'pbs플스방', '은평게임랜드', '에이원게임랜드', '디지털게임랜드', '공룡게임장', '에이스게임장',
       '가락오션게임랜드', '백악관 게임랜드', '메타버스게임넷', '우리게임장', '로얄 게임랜드', '앗싸게임랜드',
       '동문게임랜드', '마장게임랜드', '8가 게임랜드', '번성 게임랜드', '파라오', '신설동 게임랜드',
       '명보게임장', '천지산업', '천지관광호텔오락실', '산교 게임랜드', '스마일오락실', '전풍게임랜드', '황금성',
       '두꺼비 게

In [181]:
df_1_5_new = df_1_5.groupby(['법정동']).count()[['자치구']].reset_index().rename(columns = {'자치구':'성인게임업소수'})
df_1_5_new

,법정동,성인게임업소수
0,가락동,5
1,가리봉동,11
2,가산동,5
3,갈현동,3
4,개봉동,2
...,...,...
76,합정동,1
77,홍제동,1
78,화곡동,14
79,황학동,2


In [182]:
# df_1_5.to_csv('1_4_성인게임방(일반게임제공업)(좌표포함).csv', index = False)

In [183]:
# df_1_5_new.to_csv('1_4_성인게임방(일반게임제공업)수.csv', index = False)

#### 1_6 행정동별 숙박업소 수

In [184]:
df_1_6 = pd.read_csv('../data/raw_data/가중치선정/서울시 숙박업 인허가 정보.csv', encoding = 'ansi')
df_1_6

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,조건부허가신고사유,조건부허가시작일자,조건부허가종료일자,건물소유구분명,세탁기수,여성종사자수,남성종사자수,회수건조수,침대수,다중이용업소여부
0,3010000,3010000-214-2021-00001,2021-01-15,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3010000,3010000-201-2019-00008,2019-08-02,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3010000,3010000-214-2023-00002,2023-03-13,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3010000,3010000-201-2017-00010,2017-04-26,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3010000,3010000-201-2017-00011,2017-04-26,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6709,3000000,3000000-201-1976-00193,19760209,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6710,3190000,3190000-201-1991-00086,19911122,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6711,3070000,3070000-201-2002-00006,20020918,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6712,3220000,3220000-201-2005-00001,20050322,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
df_1_6 = pre_DF(df_1_6)
df_1_6

,지번주소,사업장명,업태구분명,좌표정보(X),좌표정보(Y),자치구,법정동
0,서울특별시 중구 인현동1가 158,어반스테이 명동,숙박업(생활),199345.466097,451233.369781,중구,인현동1가
1,서울특별시 중구 남산동2가 35-2,오요5(oYo5),관광호텔,198591.133748,450740.353160,중구,남산동2가
2,서울특별시 중구 남대문로3가 94,더스테이클래식호텔,숙박업(생활),198037.338494,451010.546630,중구,남대문로3가
3,서울특별시 중구 남산동2가 33-3 1~2층,오요5롯지A(oYo5LodgeA),숙박업 기타,198602.119250,450758.364667,중구,남산동2가
4,"서울특별시 중구 남산동2가 33 1,2층",오요5롯지B(oYo5LodgeB),관광호텔,198607.670947,450770.829936,중구,남산동2가
...,...,...,...,...,...,...,...
2454,서울특별시 종로구 종로5가 66-1,이녹(李綠),여관업,200161.563979,452132.172041,종로구,종로5가
2455,서울특별시 동작구 신대방동 687-8,신라장,여관업,191832.871758,442881.349005,동작구,신대방동
2456,서울특별시 성북구 정릉동 408-11,웨이브,여관업,200850.193897,456070.122190,성북구,정릉동
2457,서울특별시 강남구 대치동 995-14 지상3층~21.23층,파크하얏트서울,관광호텔,205601.904042,445151.509505,강남구,대치동


In [186]:
df_1_6['업태구분명'].unique()

array(['숙박업(생활)', '관광호텔', '숙박업 기타', '여인숙업', '여관업', '일반호텔', '휴양콘도미니엄업'],
      dtype=object)

In [187]:
df_1_6[df_1_6['업태구분명'] == '휴양콘도미니엄업']

,지번주소,사업장명,업태구분명,좌표정보(X),좌표정보(Y),자치구,법정동
163,서울특별시 서대문구 대현동 90-36,이화블로섬하우스,휴양콘도미니엄업,195013.064831,450580.617233,서대문구,대현동
247,서울특별시 서대문구 대현동 101-9,비욘드 스위트(BEYOND SUITE),휴양콘도미니엄업,194928.380121,450562.353003,서대문구,대현동
468,"서울특별시 서대문구 대현동 90-37 4,5,6,7,8층",디케이하우스,휴양콘도미니엄업,195004.191374,450587.216691,서대문구,대현동
476,서울특별시 서대문구 대현동 104-5,신촌 에버에이트 레지던스호텔,휴양콘도미니엄업,194855.910937,450573.690449,서대문구,대현동
1433,서울특별시 강북구 우이동 346 파라스파라 서울,(주)정상북한산리조트(파라스파라서울),휴양콘도미니엄업,200704.539774,462416.392839,강북구,우이동


In [188]:
df_1_6_new = df_1_6.groupby(['법정동']).count()[['자치구']].reset_index().rename(columns = {'자치구':'숙박업수'})
df_1_6_new

,법정동,숙박업수
0,가락동,4
1,가리봉동,28
2,가산동,28
3,갈월동,7
4,갈현동,7
...,...,...
286,효제동,11
287,후암동,2
288,휘경동,10
289,흑석동,3


In [189]:
# df_1_6.to_csv('1_5_숙박업(좌표포함).csv', index = False)

In [190]:
# df_1_6_new.to_csv('1_5_숙박업수.csv', index=False)

### 2. 복지시설 수 관련 전처리

In [191]:
df_2 = pd.read_csv('../data/raw_data/가중치선정/서울시 시설물 정보.csv', encoding = 'ansi')
df_2.head()

,아이디,새주소 아이디,시설 아이디,시설명,위도,경도,건물군여부,시설용도분류,소재지 도로명주소,소재지 지번주소,기타,국가지점번호,데이터 기준일자
0,3007,NaN,P0877,꿈동산어린이공원,37.650288,127.036860,BG_AA,FU_BI,서울특별시 도봉구 도봉로110나길 99,서울특별시 도봉구 창동 663-1,NaN,다사 5914 6130,2022-12-07 00:00:00.0
1,3008,NaN,P0878,미화어린이공원,37.654945,127.039690,BG_AA,FU_BI,서울특별시 도봉구 도봉로 564,서울특별시 도봉구 창동 715-17,NaN,다사 5939 6181,2022-12-07 00:00:00.0
2,3009,NaN,P0879,꿈동산어린이공원,37.573290,127.045840,BG_AA,FU_BI,서울특별시 동대문구 황물로 42,서울특별시 동대문구 답십리동 1002-1,NaN,다사 5989 5275,2022-12-07 00:00:00.0
3,3010,NaN,P0880,새움어린이공원,37.570760,127.048676,BG_AA,FU_BI,서울특별시 동대문구 천호대로55길 11,서울특별시 동대문구 답십리동 1009-1,NaN,다사 6014 5247,2022-12-07 00:00:00.0
4,3012,NaN,P0882,동산어린이공원,37.566967,127.057540,BG_AA,FU_BI,서울특별시 동대문구 전농로2나길 7,서울특별시 동대문구 답십리동 998-1,NaN,다사 6092 5204,2022-12-07 00:00:00.0


In [192]:
df_2.info() # 지번주소는 null값이 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3606 entries, 0 to 3605
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   아이디        3606 non-null   int64  
 1   새주소 아이디    0 non-null      float64
 2   시설 아이디     3606 non-null   object 
 3   시설명        3606 non-null   object 
 4   위도         3606 non-null   float64
 5   경도         3606 non-null   float64
 6   건물군여부      2260 non-null   object 
 7   시설용도분류     3606 non-null   object 
 8   소재지 도로명주소  3494 non-null   object 
 9   소재지 지번주소   3606 non-null   object 
 10  기타         89 non-null     object 
 11  국가지점번호     3606 non-null   object 
 12  데이터 기준일자   3606 non-null   object 
dtypes: float64(3), int64(1), object(9)
memory usage: 366.4+ KB


In [193]:
addr_maker(df_2, df_2['소재지 지번주소'].str.split())
df_2

,아이디,새주소 아이디,시설 아이디,시설명,위도,경도,건물군여부,시설용도분류,소재지 도로명주소,소재지 지번주소,기타,국가지점번호,데이터 기준일자,자치구,법정동
0,3007,NaN,P0877,꿈동산어린이공원,37.650288,127.036860,BG_AA,FU_BI,서울특별시 도봉구 도봉로110나길 99,서울특별시 도봉구 창동 663-1,NaN,다사 5914 6130,2022-12-07 00:00:00.0,도봉구,창동
1,3008,NaN,P0878,미화어린이공원,37.654945,127.039690,BG_AA,FU_BI,서울특별시 도봉구 도봉로 564,서울특별시 도봉구 창동 715-17,NaN,다사 5939 6181,2022-12-07 00:00:00.0,도봉구,창동
2,3009,NaN,P0879,꿈동산어린이공원,37.573290,127.045840,BG_AA,FU_BI,서울특별시 동대문구 황물로 42,서울특별시 동대문구 답십리동 1002-1,NaN,다사 5989 5275,2022-12-07 00:00:00.0,동대문구,답십리동
3,3010,NaN,P0880,새움어린이공원,37.570760,127.048676,BG_AA,FU_BI,서울특별시 동대문구 천호대로55길 11,서울특별시 동대문구 답십리동 1009-1,NaN,다사 6014 5247,2022-12-07 00:00:00.0,동대문구,답십리동
4,3012,NaN,P0882,동산어린이공원,37.566967,127.057540,BG_AA,FU_BI,서울특별시 동대문구 전농로2나길 7,서울특별시 동대문구 답십리동 998-1,NaN,다사 6092 5204,2022-12-07 00:00:00.0,동대문구,답십리동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3601,1894,NaN,B1895,연희연세치과의원,37.562363,126.927890,BG_AA,FU_BE,서울특별시 서대문구 연희로 42,서울특별시 서대문구 연희동 353-100,NaN,다사 4947 5159,2022-12-07 00:00:00.0,서대문구,연희동
3602,1908,NaN,B1909,바르다권치과의원,37.483240,126.905080,BG_AA,FU_BE,서울특별시 관악구 조원로 24,서울특별시 관악구 신림동 1653-6,NaN,다사 4740 4283,2022-12-07 00:00:00.0,관악구,신림동
3603,3398,NaN,P1268,푸른어린이공원,37.511242,126.837870,BG_AA,FU_BI,서울특별시 양천구 신정로11길 63,서울특별시 양천구 신정동 1283-1,NaN,다사 4148 4597,2022-12-07 00:00:00.0,양천구,신정동
3604,3454,NaN,P1324,돌모루어린이공원,37.542930,126.968460,BG_AA,FU_BI,서울특별시 용산구 원효로97길 50,서울특별시 용산구 청파동3가 118-102,NaN,다사 5304 4942,2022-12-07 00:00:00.0,용산구,청파동3가


In [194]:
print(df_2['자치구'].nunique(), df_2['자치구'].unique())
print(df_2['법정동'].nunique(), df_2['법정동'].unique())

25 ['도봉구' '동대문구' '강동구' '중구' '강남구' '강북구' '강서구' '용산구' '관악구' '구로구' '광진구' '금천구'
 '영등포구' '송파구' '종로구' '서대문구' '중랑구' '마포구' '노원구' '동작구' '서초구' '은평구' '성동구' '성북구'
 '양천구']
344 ['창동' '답십리동' '신설동' '명일동' '을지로7가' '강일동' '상일동' '고덕동' '대치동' '둔촌동' '암사동' '번동'
 '우이동' '가양동' '마곡동' '방화동' '개포동' '수서동' '효창동' '역삼동' '청담동' '화곡동' '삼성동' '압구정동'
 '신림동' '개봉동' '고척동' '구로동' '자양동' '항동' '독산동' '문래동3가' '잠실동' '신천동' '명륜2가'
 '충정로2가' '홍제동' '문정동' '망우동' '연남동' '남현동' '봉천동' '군자동' '광장동' '구의동' '방학동'
 '신도림동' '전농동' '장안동' '성내동' '망원동' '신촌동' '대현동' '가산동' '시흥동' '중계동' '미아동' '수유동'
 '등촌동' '염창동' '공릉동' '사당동' '제기동' '이문동' '회기동' '용두동' '청량리동' '신대방동' '도화동'
 '신공덕동' '노고산동' '예지동' '반포동' '성산동' '서교동' '창전동' '창천동' '연희동' '불광동' '서초동' '양재동'
 '방배동' '을지로6가' '신당동' '창신동' '자곡동' '도곡동' '신사동' '능동' '중곡동' '오류동' '궁동' '성수동1가'
 '상왕십리동' '옥수동' '사근동' '행당동' '마장동' '용답동' '금호동1가' '길음동' '성북동' '동선동4가' '관훈동'
 '청운동' '부암동' '서린동' '동선동1가' '돈암동' '종암동' '하월곡동' '송파동' '가락동' '마천동' '방이동'
 '숭인동' '세종로' '남대문로3가' '서소문동' '오금동' '목동' '영등포동' '월계동' '한강로2가' '영등포동4가'
 '노량진동' '당산동2가' '상수동' '쌍문동' '충정로1가' '정동' '태평로

In [195]:
df_2['기타'].unique()

array([nan, '공사중', '라모도쇼핑몰이 에이피엠 플레이스로 변경됨', '폐점', '건물 노후화로인한 폐쇄', '중복시설',
       '건물 영업종료됨', '건물공사중', '50주년 기념관 2층 박물관이나 현재 미운영', '상호명 변경으로인한 중복',
       '개인관람객은 주말, 공휴일만 이용가능 평일은 단체예약 관람객만 가능', '월요일 휴무',
       '전시종료로 인해 자택으로 사용중', '은봉관 2층 일부 아정미술관', '군사시설', '현재 리모델림 공사 중',
       '언덕길 사이에 위치', '중앙 출입구는 대형현수막 설치로 조사X', '공사로인해 2-3층 일부만 개방',
       '영업 안함', '시설 공사로인해 휴관중', '2022-11-11 개관', '숭인공원과 중복', '철거공사중',
       '코로나 종식이전 폐쇄', '홍대입구로 이전함.', '공사 중 조사불가', '현재 건물 리모델링 공사 중',
       '서울 탑 재활병원으로 명칭변경(요양병원 동일)', '폐업', '일대 전체가 재개발 구역으로 출입 불가',
       '정문공사중', '대형 상가', '사진촬영거부', '기존 주차장이 선별진료소로 사용중이라 차량 출입구 촬영 불가',
       '영등포,구청과 보건소가 한 개임', '실제 건물 모양과 스마트맵 건물모양이 다름', '아파트 단지 내 출입가능',
       '2022 12월11일까지 공사중', '시설 재정비로 공사중', '공사 중 (2021.12-2023.06)',
       '출입불가', '공사중(23년 개관예정)', '재건축으로 인한 공원철거', '공사중(조사불가)', '폐쇄',
       '공원사라짐', '오피스텔 상가 주상복합'], dtype=object)

In [196]:
# 중복 및 폐점한 리스트
closed_lst = ['폐점', '건물 노후화로인한 폐쇄', '건물 영업종료됨', '50주년 기념관 2층 박물관이나 현재 미운영', '전시종료로 인해 자택으로 사용중', 
              '영업 안함', '철거공사중', '코로나 종식이전 폐쇄', '홍대입구로 이전함.', '폐업', '재건축으로 인한 공원철거', '폐쇄', '공원사라짐',
              '중복시설', '상호명 변경으로인한 중복', '숭인공원과 중복']

In [197]:
df_2 = df_2[~df_2['기타'].isin(closed_lst)]
df_2.head()

,아이디,새주소 아이디,시설 아이디,시설명,위도,경도,건물군여부,시설용도분류,소재지 도로명주소,소재지 지번주소,기타,국가지점번호,데이터 기준일자,자치구,법정동
0,3007,NaN,P0877,꿈동산어린이공원,37.650288,127.036860,BG_AA,FU_BI,서울특별시 도봉구 도봉로110나길 99,서울특별시 도봉구 창동 663-1,NaN,다사 5914 6130,2022-12-07 00:00:00.0,도봉구,창동
1,3008,NaN,P0878,미화어린이공원,37.654945,127.039690,BG_AA,FU_BI,서울특별시 도봉구 도봉로 564,서울특별시 도봉구 창동 715-17,NaN,다사 5939 6181,2022-12-07 00:00:00.0,도봉구,창동
2,3009,NaN,P0879,꿈동산어린이공원,37.573290,127.045840,BG_AA,FU_BI,서울특별시 동대문구 황물로 42,서울특별시 동대문구 답십리동 1002-1,NaN,다사 5989 5275,2022-12-07 00:00:00.0,동대문구,답십리동
3,3010,NaN,P0880,새움어린이공원,37.570760,127.048676,BG_AA,FU_BI,서울특별시 동대문구 천호대로55길 11,서울특별시 동대문구 답십리동 1009-1,NaN,다사 6014 5247,2022-12-07 00:00:00.0,동대문구,답십리동
4,3012,NaN,P0882,동산어린이공원,37.566967,127.057540,BG_AA,FU_BI,서울특별시 동대문구 전농로2나길 7,서울특별시 동대문구 답십리동 998-1,NaN,다사 6092 5204,2022-12-07 00:00:00.0,동대문구,답십리동


In [198]:
df_2.drop(columns = ['아이디', '새주소 아이디', '시설 아이디', '건물군여부', '소재지 도로명주소', '국가지점번호', '데이터 기준일자', '기타'], inplace = True)
df_2.head()

C:\Users\yju12\AppData\Local\Temp\ipykernel_3600\4236719313.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2.drop(columns = ['아이디', '새주소 아이디', '시설 아이디', '건물군여부', '소재지 도로명주소', '국가지점번호', '데이터 기준일자', '기타'], inplace = True)


,시설명,위도,경도,시설용도분류,소재지 지번주소,자치구,법정동
0,꿈동산어린이공원,37.650288,127.036860,FU_BI,서울특별시 도봉구 창동 663-1,도봉구,창동
1,미화어린이공원,37.654945,127.039690,FU_BI,서울특별시 도봉구 창동 715-17,도봉구,창동
2,꿈동산어린이공원,37.573290,127.045840,FU_BI,서울특별시 동대문구 답십리동 1002-1,동대문구,답십리동
3,새움어린이공원,37.570760,127.048676,FU_BI,서울특별시 동대문구 답십리동 1009-1,동대문구,답십리동
4,동산어린이공원,37.566967,127.057540,FU_BI,서울특별시 동대문구 답십리동 998-1,동대문구,답십리동


In [199]:
df_2_1 = df_2[df_2['시설용도분류'] == 'FU_BA'].reset_index(drop = True) # 대규모 점포
df_2_2 = df_2[df_2['시설용도분류'] == 'FU_BB'].reset_index(drop = True) # 도서관
df_2_3 = df_2[df_2['시설용도분류'] == 'FU_BC'].reset_index(drop = True) # 미술관, 박물관, 극장
df_2_4 = df_2[df_2['시설용도분류'].isin(['FU_BD', 'FU_BE', 'FU_BG', 'FU_BH'])].reset_index(drop = True) # 병원, 의원, 보건소, 보건지소
df_2_5 = df_2[df_2['시설용도분류'] == 'FU_BI'].reset_index(drop = True) # 공원
df_2_6 = df_2[df_2['시설용도분류'] == 'FU_BJ'].reset_index(drop = True) # 하천

In [200]:
display(df_2_1.head(), df_2_2.head(), df_2_3.head(), df_2_4.head(), df_2_5.head(), df_2_6.head())

,시설명,위도,경도,시설용도분류,소재지 지번주소,자치구,법정동
0,주양쇼핑,37.552520,127.153496,FU_BA,서울특별시 강동구 명일동 48,강동구,명일동
1,에이피엠 플레이스,37.565273,127.008410,FU_BA,서울특별시 중구 을지로7가 105,중구,을지로7가
2,대치2단지상가,37.493477,127.075790,FU_BA,서울특별시 강남구 개포동 12,강남구,개포동
3,로즈데일오피스텔,37.487328,127.103140,FU_BA,서울특별시 강남구 수서동 724,강남구,수서동
4,롯데백화점 강남점,37.497060,127.053600,FU_BA,서울특별시 강남구 대치동 937,강남구,대치동


,시설명,위도,경도,시설용도분류,소재지 지번주소,자치구,법정동
0,송파어린이도서관,37.512110,127.08242,FU_BB,서울특별시 송파구 잠실동 19-6,송파구,잠실동
1,하상시각장애인도서관,37.492550,127.07510,FU_BB,서울특별시 강남구 개포동 12-5,강남구,개포동
2,도곡정보문화도서관,37.488308,127.03887,FU_BB,서울특별시 강남구 도곡동 892-6,강남구,도곡동
3,즐거운도서관,37.499740,127.05785,FU_BB,서울특별시 강남구 대치동 1011-66,강남구,대치동
4,중곡문화체육센터도서관,37.567800,127.08473,FU_BB,서울특별시 광진구 중곡동 168-8,광진구,중곡동


,시설명,위도,경도,시설용도분류,소재지 지번주소,자치구,법정동
0,김세중미술관,37.542038,126.963875,FU_BC,서울특별시 용산구 효창동 5-390,용산구,효창동
1,롯데뮤지엄,37.512570,127.102570,FU_BC,서울특별시 송파구 신천동 29,송파구,신천동
2,짚풀생활사박물관,37.584908,126.999990,FU_BC,서울특별시 종로구 명륜2가 8-4,종로구,명륜2가
3,도산안창호기념관,37.523964,127.035590,FU_BC,서울특별시 강남구 신사동 649-9,강남구,신사동
4,경기여자고등학교 경운박물관,37.487410,127.066280,FU_BC,서울특별시 강남구 개포동 152,강남구,개포동


,시설명,위도,경도,시설용도분류,소재지 지번주소,자치구,법정동
0,서울중앙요양병원,37.565200,126.965460,FU_BD,서울특별시 서대문구 충정로2가 78,서대문구,충정로2가
1,제이여성병원,37.588520,126.944916,FU_BD,서울특별시 서대문구 홍제동 251-4,서대문구,홍제동
2,의료법인일맥의료재단 송파인애가한방병원,37.487290,127.129486,FU_BD,서울특별시 송파구 문정동 17-5,송파구,문정동
3,마인드웰병원,37.599556,127.096930,FU_BD,서울특별시 중랑구 망우동 489-1,중랑구,망우동
4,조앤조병원,37.600273,127.104710,FU_BD,서울특별시 중랑구 망우동 207-1,중랑구,망우동


,시설명,위도,경도,시설용도분류,소재지 지번주소,자치구,법정동
0,꿈동산어린이공원,37.650288,127.036860,FU_BI,서울특별시 도봉구 창동 663-1,도봉구,창동
1,미화어린이공원,37.654945,127.039690,FU_BI,서울특별시 도봉구 창동 715-17,도봉구,창동
2,꿈동산어린이공원,37.573290,127.045840,FU_BI,서울특별시 동대문구 답십리동 1002-1,동대문구,답십리동
3,새움어린이공원,37.570760,127.048676,FU_BI,서울특별시 동대문구 답십리동 1009-1,동대문구,답십리동
4,동산어린이공원,37.566967,127.057540,FU_BI,서울특별시 동대문구 답십리동 998-1,동대문구,답십리동


,시설명,위도,경도,시설용도분류,소재지 지번주소,자치구,법정동
0,청계천,37.569668,127.005745,FU_BJ,서울특별시 종로구 서린동 148,종로구,서린동
1,탄천,37.498432,127.088290,FU_BJ,서울특별시 강남구 일원동 445,강남구,일원동
2,양재천,37.488235,127.057540,FU_BJ,서울특별시 강남구 도곡동 548,강남구,도곡동
3,안양천,37.456050,126.892100,FU_BJ,서울특별시 금천구 독산동 770-4,금천구,독산동
4,홍제천,37.581017,126.937454,FU_BJ,서울특별시 마포구 성산동 587-4,마포구,성산동


- 법정동 별 groupby

In [201]:
df_2_shop = df_2_1.groupby(['법정동']).count()[['자치구']].reset_index().rename(columns = {'자치구':'대형쇼핑몰수'})
df_2_lib = df_2_2.groupby(['법정동']).count()[['자치구']].reset_index().rename(columns = {'자치구':'도서관 수'})
df_2_com = df_2_3.groupby(['법정동']).count()[['자치구']].reset_index().rename(columns = {'자치구':'미술관, 박물관, 극장수'})
df_2_med = df_2_4.groupby(['법정동']).count()[['자치구']].reset_index().rename(columns = {'자치구':'의료시설 수'})
df_2_prk = df_2_5.groupby(['법정동']).count()[['자치구']].reset_index().rename(columns = {'자치구':'공원 수'})
df_2_rvr = df_2_6.groupby(['법정동']).count()[['자치구']].reset_index().rename(columns = {'자치구':'하천 수'})

In [202]:
display(df_2_shop, df_2_lib, df_2_com, df_2_med, df_2_prk, df_2_rvr)
# 하천수는 8개 밖에 없어 제외해야 할 듯함.

,법정동,대형쇼핑몰수
0,가락동,5
1,가산동,7
2,가양동,6
3,갈현동,2
4,강일동,1
...,...,...
187,황학동,1
188,회기동,1
189,회현동1가,1
190,흑석동,1


,법정동,도서관 수
0,가락동,1
1,가리봉동,1
2,가산동,1
3,가양동,1
4,강일동,1
...,...,...
131,홍제동,1
132,화곡동,4
133,화동,1
134,후암동,2


,법정동,"미술관, 박물관, 극장수"
0,가락동,1
1,가양동,2
2,가회동,3
3,개포동,1
4,견지동,1
...,...,...
103,화동,2
104,화양동,1
105,회기동,1
106,회현동1가,1


,법정동,의료시설 수
0,가락동,10
1,가리봉동,5
2,가산동,6
3,갈현동,4
4,강일동,1
...,...,...
186,황학동,2
187,회기동,2
188,휘경동,2
189,흑석동,1


,법정동,공원 수
0,가락동,17
1,가산동,3
2,가양동,13
3,갈현동,6
4,강일동,7
...,...,...
241,황학동,1
242,효창동,2
243,후암동,1
244,휘경동,4


,법정동,하천 수
0,도곡동,1
1,독산동,1
2,면목동,1
3,북가좌동,1
4,서린동,1
5,성산동,1
6,여의도동,1
7,일원동,1


In [203]:
# df_2_1.to_csv('2_1_대형쇼핑몰(좌표포함).csv', index=False)
# df_2_2.to_csv('2_2_도서관(좌표포함).csv', index=False)
# df_2_3.to_csv('2_3_미술관 박물관 극장(좌표포함).csv', index=False)
# df_2_4.to_csv('2_4_의료시설(병원,의원,보건소,보건지소)(좌표포함).csv', index=False)
# df_2_5.to_csv('2_5_공원(좌표포함).csv', index=False)

In [204]:
# df_2_shop.to_csv('2_1_대형쇼핑몰수.csv', index=False)
# df_2_lib.to_csv('2_2_도서관수.csv', index=False)
# df_2_com.to_csv('2_3_미술관 박물관 극장수.csv', index=False)
# df_2_med.to_csv('2_4_의료시설(병원,의원,보건소,보건지소)수.csv', index=False)
# df_2_prk.to_csv('2_5_공원수.csv', index=False)

### 3. 행정동별 CCTV 설치 수 전처리

In [205]:
df_3 = pd.read_csv('../data/raw_data/가중치선정/서울시 안심이 CCTV 연계 현황.csv', encoding = 'ansi')
df_3.head()

,자치구,안심 주소,위도,경도,CCTV 수량,수정 일시
0,중랑구,중화2동 325-103 (공원 입구)_C-JH02-0035,37.5958,127.0709,1,2022-12-01
1,중랑구,신내1동 819(고정2)_C-SN01-0029-B,37.6135,127.1113,1,2022-12-01
2,중랑구,중화2동 325-103(고정1)_C-JH02-0035-A,37.5958,127.0709,1,2022-12-01
3,중랑구,신내1동 819(고정1)_C-SN01-0029-A,37.6135,127.1113,1,2022-12-01
4,중랑구,중화2동 325-103(고정2)_C-JH02-0035-B,37.5958,127.0709,1,2022-12-01


In [206]:
# 안심주소의 표기가 자치구별로 다름...

for gu in df_3['자치구'].unique():
    display(df_3[df_3['자치구'] == gu].head(3))

,자치구,안심 주소,위도,경도,CCTV 수량,수정 일시
0,중랑구,중화2동 325-103 (공원 입구)_C-JH02-0035,37.5958,127.0709,1,2022-12-01
1,중랑구,신내1동 819(고정2)_C-SN01-0029-B,37.6135,127.1113,1,2022-12-01
2,중랑구,중화2동 325-103(고정1)_C-JH02-0035-A,37.5958,127.0709,1,2022-12-01


,자치구,안심 주소,위도,경도,CCTV 수량,수정 일시
3471,중구,"남407보2(남대문시장8길 2,남창동 31-1,국민은행 삼거리)",37.5604,126.9786,1,2022-12-01
3472,중구,"남407보1(남대문시장8길 2,남창동 31-1,국민은행 삼거리)",37.5604,126.9786,1,2022-12-01
3473,중구,"남407(남대문시장8길 2,남창동 31-1,국민은행 삼거리)",37.5604,126.9786,1,2022-12-01


,자치구,안심 주소,위도,경도,CCTV 수량,수정 일시
5807,종로구,방범045_백석동길 232_메인,37.5981,126.9671,1,2022-12-01
5808,종로구,방범477_세검정로7나길 40_검지2,37.6037,126.9595,1,2022-12-01
5809,종로구,방범477_세검정로7나길 40_메인,37.6037,126.9595,1,2022-12-01


,자치구,안심 주소,위도,경도,CCTV 수량,수정 일시
7737,은평구,녹번동 35-96 GS25 앞 /1,37.6037,126.9347,1,2022-12-01
7738,은평구,녹번동 45-7 은평초교 옆길(어린이) /2,37.6013,126.9338,1,2022-12-01
7739,은평구,녹번동 71-11 산골마을 /4,37.5991,126.9387,1,2022-12-01


,자치구,안심 주소,위도,경도,CCTV 수량,수정 일시
11848,용산구,자치03-07(A1) 21-1차(후암로57길 17-32),37.5524,126.9751,1,2022-12-01
11849,용산구,"자치08-37(00) 21-2차(도원동 8-24, CU편의점 앞)",37.5394,126.9581,1,2022-12-01
11850,용산구,"자치09-15(00) (한강로3가 40-946, 마사회 뒷길)",37.5274,126.9631,1,2022-12-01


...

,자치구,안심 주소,위도,경도,CCTV 수량,수정 일시
62102,관악구,A9006 1-1RZ 새숲공원 (조원동 539-14와 539-13),37.4825,126.9130,1,2022-12-01
62103,관악구,A9011 1-1RZ 태양공원 (대학동 1542-1와 1542-2),37.4684,126.9366,1,2022-12-01
62104,관악구,A9012 1-GS 낙성대공원 관리사무소 (낙성대동 228),37.4720,126.9597,1,2022-12-01


,자치구,안심 주소,위도,경도,CCTV 수량,수정 일시
67468,강서구,C181008(SC140) - 마곡중앙5로1길 연구단지D2 앞(보조1) (3시 롯...,37.5720,126.8267,1,2022-12-01
67469,강서구,C181008(SC140) - 마곡중앙5로1길 연구단지D2 앞(교차로1) (3시 롯...,37.5720,126.8267,1,2022-12-01
67470,강서구,C181009(SC204) - 공항대로 마곡역 건너 연결녹지22호 앞,37.5602,126.8272,1,2022-12-01


,자치구,안심 주소,위도,경도,CCTV 수량,수정 일시
70678,강북구,test,37.6398,127.0255,1,2022-12-01
70679,강북구,"[3166][방범-우이-154](회전,남동)",37.6608,127.0117,1,2022-12-01
70680,강북구,"[3163][방범-우이-144](고정,서)",37.6398,127.0255,1,2022-12-01


,자치구,안심 주소,위도,경도,CCTV 수량,수정 일시
73536,강동구,상일1동 177 까치어린이공원 (방범-상일1동-021-01),37.5477,127.1624,1,2022-12-01
73537,강동구,상일1동 177 까치어린이공원 (방범-상일1동-021-02),37.5477,127.1624,1,2022-12-01
73538,강동구,상일1동 166 한영중고교앞사거리(스쿨존-상일1동-011-02),37.5491,127.1563,1,2022-12-01


,자치구,안심 주소,위도,경도,CCTV 수량,수정 일시
76727,강남구,수서-209-00,37.4907,127.1039,1,2022-12-01
76728,강남구,개포2-220-02,37.4869,127.0601,1,2022-12-01
76729,강남구,논현1-277-00,37.5066,127.0247,1,2022-12-01


- QGIS 위,경도로 CCTV수 처리완료

In [207]:
df_3 = pd.read_csv('../data/raw_data/가중치선정/행정동별 CCTV수.csv', encoding = 'ansi')

In [208]:
df_3_new = df_3[['ADM_NM', 'NUMPOINTS']].dropna().rename(columns = {'ADM_NM':'행정동', 'NUMPOINTS':'CCTV수'})
df_3_new

,행정동,CCTV수
1,사직동,139
2,삼청동,44
3,부암동,144
4,평창동,120
5,한남동,119
...,...,...
422,가회동,55
423,종로1·2·3·4가동,227
424,방배2동,353
425,미아동,123


In [209]:
# df_3_new.to_csv('3_행정동별 CCTV수.csv', index = False)

### 4. 주유소, 경로당 등 법규관련

#### 4_1 주유소 수 

In [210]:
df_4_1 = pd.read_csv('../data/raw_data/가중치선정/서울시 석유판매업 인허가 정보.csv', encoding='ansi')
df_4_1

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,도로명우편번호,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),자본금,거래처
0,3130000,1993313011801500022,1993-11-29,NaN,1,영업/정상,6,휴지사업재개,NaN,NaN,...,NaN,주식회사타이거통상 월드컵주유소,2023-12-18 14:54:30,U,2022-11-01 22:00:00.0,주유소,191961.827164,451692.303107,NaN,NaN
1,3150000,2008315010001219911,2008-04-04,NaN,3,폐업,3,폐지,2023-03-31,NaN,...,NaN,SK그린에너지,2023-03-31 10:55:00,U,2022-12-04 00:02:00.0,일반판매소,187613.276168,450158.253529,NaN,NaN
2,3190000,2019319017701200001,2019-06-26,NaN,1,영업/정상,1,신규등록,NaN,NaN,...,6959.0,남양에너지,2023-03-10 17:20:58,U,2022-12-02 23:02:00.0,일반판매소,194076.692856,444318.185903,NaN,NaN
3,3160000,1993316007101500036,1993-07-24,NaN,1,영업/정상,6,휴지사업재개,NaN,2012-02-21,...,NaN,(주)한미석유 구로그린주유소,2023-03-27 16:41:48,U,2022-12-02 22:09:00.0,주유소,190065.327992,443794.958060,NaN,NaN
4,3230000,1992323013101500002,1992-12-21,NaN,1,영업/정상,1,신규등록,NaN,NaN,...,NaN,에이치디현대오일뱅크(주)직영 잠실셀프주유소,2023-04-07 10:02:11,U,2022-12-04 00:09:00.0,주유소,209565.447766,444507.063566,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899,3130000,2001313008001200001,1979-04-18,NaN,1,영업/정상,1,신규등록,NaN,NaN,...,NaN,영풍에너지,2023-11-09 13:36:31,U,2022-10-31 23:01:00.0,일반판매소,192433.736074,451418.064309,NaN,NaN
1900,3210000,2007321012201500001,2007-11-22,NaN,1,영업/정상,7,영업개시,NaN,NaN,...,NaN,(주)한미석유 서초주유소,2023-12-11 15:08:01,U,2022-11-01 23:03:00.0,주유소,200883.863531,442666.820810,NaN,NaN
1901,3210000,1984321007601500014,1984-10-23,NaN,1,영업/정상,7,영업개시,NaN,NaN,...,6763.0,태봉주유소,2023-12-15 17:57:13,U,2022-11-01 23:07:00.0,주유소,202422.841634,440678.229387,NaN,NaN
1902,3110000,1992311008801500021,1992-05-09,NaN,2,휴업,5,사업휴지,NaN,2020-06-01,...,NaN,삼융주유소,2024-01-09 15:06:18,U,2023-11-30 23:01:00.0,주유소,190321.180294,453524.393765,NaN,NaN


In [211]:
df_4_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1904 entries, 0 to 1903
Data columns (total 27 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   개방자치단체코드  1904 non-null   int64  
 1   관리번호      1904 non-null   int64  
 2   인허가일자     1904 non-null   object 
 3   인허가취소일자   20 non-null     float64
 4   영업상태코드    1904 non-null   int64  
 5   영업상태명     1904 non-null   object 
 6   상세영업상태코드  1904 non-null   int64  
 7   상세영업상태명   1904 non-null   object 
 8   폐업일자      705 non-null    object 
 9   휴업시작일자    160 non-null    object 
 10  휴업종료일자    160 non-null    object 
 11  재개업일자     132 non-null    object 
 12  전화번호      1823 non-null   object 
 13  소재지면적     905 non-null    float64
 14  소재지우편번호   0 non-null      float64
 15  지번주소      1893 non-null   object 
 16  도로명주소     1614 non-null   object 
 17  도로명우편번호   97 non-null     float64
 18  사업장명      1904 non-null   object 
 19  최종수정일자    1904 non-null   object 
 20  데이터갱신구분   1904 non-null   obje

In [212]:
df_4_1['영업상태명'].unique()

array(['영업/정상', '폐업', '휴업', '취소/말소/만료/정지/중지'], dtype=object)

In [213]:
df_4_1 = df_4_1[df_4_1['영업상태명'] == '영업/정상']
df_4_1

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,도로명우편번호,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),자본금,거래처
0,3130000,1993313011801500022,1993-11-29,NaN,1,영업/정상,6,휴지사업재개,NaN,NaN,...,NaN,주식회사타이거통상 월드컵주유소,2023-12-18 14:54:30,U,2022-11-01 22:00:00.0,주유소,191961.827164,451692.303107,NaN,NaN
2,3190000,2019319017701200001,2019-06-26,NaN,1,영업/정상,1,신규등록,NaN,NaN,...,6959.0,남양에너지,2023-03-10 17:20:58,U,2022-12-02 23:02:00.0,일반판매소,194076.692856,444318.185903,NaN,NaN
3,3160000,1993316007101500036,1993-07-24,NaN,1,영업/정상,6,휴지사업재개,NaN,2012-02-21,...,NaN,(주)한미석유 구로그린주유소,2023-03-27 16:41:48,U,2022-12-02 22:09:00.0,주유소,190065.327992,443794.958060,NaN,NaN
4,3230000,1992323013101500002,1992-12-21,NaN,1,영업/정상,1,신규등록,NaN,NaN,...,NaN,에이치디현대오일뱅크(주)직영 잠실셀프주유소,2023-04-07 10:02:11,U,2022-12-04 00:09:00.0,주유소,209565.447766,444507.063566,NaN,NaN
5,3040000,1992304007301500017,1992-04-20,NaN,1,영업/정상,6,휴지사업재개,NaN,2014-07-01,...,NaN,에이치디현대오일뱅크(주)직영 능동주유소,2023-04-07 11:14:29,U,2022-12-04 00:09:00.0,주유소,207216.133825,450367.951562,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1898,3040000,1992304007301500015,1992-04-20,NaN,1,영업/정상,1,신규등록,NaN,NaN,...,NaN,주식회사 유니스에너지,2023-06-19 15:35:20,U,2022-12-05 22:01:00.0,주유소,206638.326812,448433.165074,NaN,NaN
1899,3130000,2001313008001200001,1979-04-18,NaN,1,영업/정상,1,신규등록,NaN,NaN,...,NaN,영풍에너지,2023-11-09 13:36:31,U,2022-10-31 23:01:00.0,일반판매소,192433.736074,451418.064309,NaN,NaN
1900,3210000,2007321012201500001,2007-11-22,NaN,1,영업/정상,7,영업개시,NaN,NaN,...,NaN,(주)한미석유 서초주유소,2023-12-11 15:08:01,U,2022-11-01 23:03:00.0,주유소,200883.863531,442666.820810,NaN,NaN
1901,3210000,1984321007601500014,1984-10-23,NaN,1,영업/정상,7,영업개시,NaN,NaN,...,6763.0,태봉주유소,2023-12-15 17:57:13,U,2022-11-01 23:07:00.0,주유소,202422.841634,440678.229387,NaN,NaN


In [214]:
def addr_maker(df, addr_lst):
    dong_list = []
    for add in addr_lst:
        dong_list.append(add[2])

    gu_list = []
    for add in addr_lst:
        gu_list.append(add[1])


    df['자치구'] = gu_list
    df['법정동'] = dong_list

def pre_DF(df):
    df = df[['지번주소', '사업장명', '업태구분명', '좌표정보(X)', '좌표정보(Y)']]
    df = df.dropna(subset = ['지번주소']).reset_index(drop = True)

    addr_maker(df, df['지번주소'].str.split())

    return df

In [215]:
df_4_1 = pre_DF(df_4_1)
df_4_1

,지번주소,사업장명,업태구분명,좌표정보(X),좌표정보(Y),자치구,법정동
0,서울특별시 마포구 성산동 200-4,주식회사타이거통상 월드컵주유소,주유소,191961.827164,451692.303107,마포구,성산동
1,서울특별시 동작구 상도동 355-20,남양에너지,일반판매소,194076.692856,444318.185903,동작구,상도동
2,서울특별시 구로구 구로동 100-10,(주)한미석유 구로그린주유소,주유소,190065.327992,443794.958060,구로구,구로동
3,서울특별시 송파구 석촌동 295,에이치디현대오일뱅크(주)직영 잠실셀프주유소,주유소,209565.447766,444507.063566,송파구,석촌동
4,"서울특별시 광진구 능동 236-4 236-5, 236-24",에이치디현대오일뱅크(주)직영 능동주유소,주유소,207216.133825,450367.951562,광진구,능동
...,...,...,...,...,...,...,...
610,서울특별시 광진구 자양동 227-4,주식회사 유니스에너지,주유소,206638.326812,448433.165074,광진구,자양동
611,서울특별시 마포구 성산동 21-1,영풍에너지,일반판매소,192433.736074,451418.064309,마포구,성산동
612,서울특별시 서초구 서초동 1582-23,(주)한미석유 서초주유소,주유소,200883.863531,442666.820810,서초구,서초동
613,서울특별시 서초구 우면동 13-6 태봉주유소,태봉주유소,주유소,202422.841634,440678.229387,서초구,우면동


In [216]:
df_4_1_new = df_4_1.groupby(['법정동']).count()[['자치구']].reset_index().rename(columns = {'자치구':'주유소 수'})
df_4_1_new

,법정동,주유소 수
0,가락동,4
1,가리봉동,3
2,가산동,2
3,가양동,5
4,갈월동,1
...,...,...
207,화곡동,10
208,화양동,1
209,후암동,1
210,휘경동,5


In [217]:
# df_4_1.to_csv('../data/raw_data/가중치선정/완료/4_1_주유소(좌표포함).csv', index=False)
# df_4_1_new.to_csv('../data/raw_data/가중치선정/완료/4_1_주유소수.csv', index=False)

#### 4_2 가스충전소 수

In [218]:
df_4_2 = pd.read_csv('../data/raw_data/가중치선정/서울시 액화석유가스업 현황.csv', encoding='ansi')
df_4_2

,시군구코드,인허가번호,업소번호,사업종류,상호,전화번호,허가일자,주소
0,3150000,2022315020404100001,1,가스용품제조사업,(주)파커스인터내셔널,NaN,20220729,서울특별시 강서구 등촌동 684번지 2호 우리벤처타운
1,3150000,2022315000004100001,1,가스용품제조사업,(주)파커스인터내셔널,NaN,20220727,서울특별시 강서구 등촌동 684번지 2호 우리벤처타운
2,3180000,2019318022104100001,1,판매사업,동진가스,NaN,20191224,서울특별시 영등포구 도림동 187번지 80호
3,3150000,2017315015104100002,1,충전사업,방화동LPG충전소,NaN,20171204,서울특별시 강서구 개화동 565번지 4호
4,3230000,2017323023104100001,1,충전사업,복지송파충전소,02 449 9123,20170425,서울특별시 송파구 장지동 907번지
...,...,...,...,...,...,...,...,...
514,3030000,1972303007404104302,1,판매사업,고려가스상사,NaN,19720428,서울특별시 성동구 홍익동 77번지
515,3090000,1970309008704100006,1,충전사업,주식회사 행복에너지,02 34916671,19701120,서울특별시 도봉구 방학동 707번지 2호
516,3080000,1970308007504100001,1,충전사업,서울와사공업(주),02 9021112,19701120,서울특별시 강북구 번동 446번지 31호
517,3160000,1970316007104100004,1,충전사업,대성산업(주)오류동충전소,0226120039,19700224,서울특별시 구로구 오류동 74번지 8호 오류가스충전소


In [219]:
df_4_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519 entries, 0 to 518
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시군구코드   519 non-null    int64 
 1   인허가번호   519 non-null    object
 2   업소번호    519 non-null    int64 
 3   사업종류    519 non-null    object
 4   상호      519 non-null    object
 5   전화번호    274 non-null    object
 6   허가일자    519 non-null    int64 
 7   주소      494 non-null    object
dtypes: int64(3), object(5)
memory usage: 32.6+ KB


In [220]:
df_4_2.dropna(subset='주소', inplace=True)

In [221]:
addr_maker(df_4_2, df_4_2['주소'].str.split())

In [222]:
df_4_2.drop(columns=['시군구코드', '인허가번호', '업소번호', '전화번호', '허가일자'], inplace=True)
df_4_2

,사업종류,상호,주소,자치구,법정동
0,가스용품제조사업,(주)파커스인터내셔널,서울특별시 강서구 등촌동 684번지 2호 우리벤처타운,강서구,등촌동
1,가스용품제조사업,(주)파커스인터내셔널,서울특별시 강서구 등촌동 684번지 2호 우리벤처타운,강서구,등촌동
2,판매사업,동진가스,서울특별시 영등포구 도림동 187번지 80호,영등포구,도림동
3,충전사업,방화동LPG충전소,서울특별시 강서구 개화동 565번지 4호,강서구,개화동
4,충전사업,복지송파충전소,서울특별시 송파구 장지동 907번지,송파구,장지동
...,...,...,...,...,...
514,판매사업,고려가스상사,서울특별시 성동구 홍익동 77번지,성동구,홍익동
515,충전사업,주식회사 행복에너지,서울특별시 도봉구 방학동 707번지 2호,도봉구,방학동
516,충전사업,서울와사공업(주),서울특별시 강북구 번동 446번지 31호,강북구,번동
517,충전사업,대성산업(주)오류동충전소,서울특별시 구로구 오류동 74번지 8호 오류가스충전소,구로구,오류동


In [223]:
df_4_2_new = df_4_2.groupby(['법정동']).count()[['자치구']].reset_index().rename(columns = {'자치구':'가스충전소 수'})
df_4_2_new

,법정동,가스충전소 수
0,가리봉동,1
1,가산동,3
2,가양동,2
3,갈현동,3
4,강일동,1
...,...,...
190,화양동,1
191,황학동,1
192,회현동1가,1
193,휘경동,1


In [224]:
# df_4_2.to_csv('../data/raw_data/가중치선정/완료/4_2_가스충전소(주소).csv', index=False)

In [225]:
# df_4_2_new.to_csv('../data/raw_data/가중치선정/완료/4_2_가스충전소수.csv', index=False)

#### 4_3 경로당 수

In [226]:
df_4_3 = pd.read_csv('../data/raw_data/가중치선정/서울시 경로당 정보.csv', encoding='ansi')
df_4_3

,번호,사업장명,소재지전체주소,도로명전체주소,인허가일자,영업상태명,폐업일자,휴업시작일자,휴업종료일자,재개업일자,소재지면적,소재지우편번호,입소정원,자격소유인원수,총인원수,인허가번호,상세영업상태명
0,3745,정릉꿈에그린(아)경로당,서울특별시 성북구 정릉동 정릉꿈에그린,NaN,20170808,운영중,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3070000201800004,운영
1,3744,구파발10단 제3경로당,서울특별시 은평구 진관동 은평뉴타운구파발,NaN,20121212,운영중,NaN,NaN,NaN,NaN,NaN,NaN,22.0,NaN,NaN,3110000201800003,운영
2,3743,아현 아이파크 경로당,서울특별시 마포구 아현동 104동,NaN,20180406,운영중,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN,NaN,3130000201800004,운영
3,3742,상암월드컵파크12단지(아)경로당,서울특별시 마포구 상암동 상암월드컵파크12단지,NaN,20110414,운영중,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3130000201800014,운영
4,3741,상암월드컵파크11단지 경로당,서울특별시 마포구 상암동 상암월드컵파크11단지,NaN,20121024,운영중,NaN,NaN,NaN,NaN,NaN,NaN,36.0,NaN,NaN,3130000201800013,운영
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3740,5,마곡엠밸리6단지경로당,서울특별시 강서구 마곡동,NaN,20160801,운영중,NaN,NaN,NaN,NaN,NaN,NaN,58.0,NaN,NaN,3150000201600015,운영
3741,4,큰덕경로당,서울특별시 마포구 공덕동,NaN,20111014,운영중,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,3130000201600006,운영
3742,3,구립양평3가경로당,서울특별시 영등포구 양평동3가 36번지,NaN,19890706,운영중,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,3180000201500006,운영
3743,2,백련산힐스테이트1차아파트경로당,서울특별시 은평구 응암동,NaN,20131121,운영중,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,3110000201500015,운영


In [227]:
df_4_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3745 entries, 0 to 3744
Data columns (total 17 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   번호       3745 non-null   int64  
 1   사업장명     3745 non-null   object 
 2   소재지전체주소  3745 non-null   object 
 3   도로명전체주소  0 non-null      float64
 4   인허가일자    3745 non-null   int64  
 5   영업상태명    3745 non-null   object 
 6   폐업일자     0 non-null      float64
 7   휴업시작일자   0 non-null      float64
 8   휴업종료일자   0 non-null      float64
 9   재개업일자    20 non-null     float64
 10  소재지면적    0 non-null      float64
 11  소재지우편번호  0 non-null      float64
 12  입소정원     3687 non-null   float64
 13  자격소유인원수  3279 non-null   float64
 14  총인원수     3295 non-null   float64
 15  인허가번호    3745 non-null   int64  
 16  상세영업상태명  3717 non-null   object 
dtypes: float64(10), int64(3), object(4)
memory usage: 497.5+ KB


In [228]:
df_4_3['상세영업상태명'].unique()

array(['운영', nan], dtype=object)

In [229]:
df_4_3.dropna(subset='상세영업상태명', inplace=True)

In [230]:
df_4_3['소재지전체주소'].str.split()

0              [서울특별시, 성북구, 정릉동, 정릉꿈에그린]
1            [서울특별시, 은평구, 진관동, 은평뉴타운구파발]
2                [서울특별시, 마포구, 아현동, 104동]
3         [서울특별시, 마포구, 상암동, 상암월드컵파크12단지]
4         [서울특별시, 마포구, 상암동, 상암월드컵파크11단지]
                      ...               
3740                   [서울특별시, 강서구, 마곡동]
3741                   [서울특별시, 마포구, 공덕동]
3742          [서울특별시, 영등포구, 양평동3가, 36번지]
3743                   [서울특별시, 은평구, 응암동]
3744    [서울특별시, 은평구, 응암동, 백련산힐스테이트2차아파트]
Name: 소재지전체주소, Length: 3717, dtype: object

In [231]:
# for i,add in enumerate(df_4_3['소재지전체주소'].str.split()):
#     print(i,add[2])

In [232]:
# 주소가 누락된 부분은 검색해서 채워넣음 
df_4_3.loc[df_4_3['소재지전체주소'].str.len() < 13, '소재지전체주소'] = '서울특별시 강서구 염창동 292번지'

In [233]:
df_4_3.loc[df_4_3['소재지전체주소'].str.len() < 13]

,번호,사업장명,소재지전체주소,도로명전체주소,인허가일자,영업상태명,폐업일자,휴업시작일자,휴업종료일자,재개업일자,소재지면적,소재지우편번호,입소정원,자격소유인원수,총인원수,인허가번호,상세영업상태명


In [234]:
addr_maker(df_4_3, df_4_3['소재지전체주소'].str.split())

In [235]:
df_4_3 = df_4_3[['사업장명', '소재지전체주소', '자치구', '법정동']]
df_4_3

,사업장명,소재지전체주소,자치구,법정동
0,정릉꿈에그린(아)경로당,서울특별시 성북구 정릉동 정릉꿈에그린,성북구,정릉동
1,구파발10단 제3경로당,서울특별시 은평구 진관동 은평뉴타운구파발,은평구,진관동
2,아현 아이파크 경로당,서울특별시 마포구 아현동 104동,마포구,아현동
3,상암월드컵파크12단지(아)경로당,서울특별시 마포구 상암동 상암월드컵파크12단지,마포구,상암동
4,상암월드컵파크11단지 경로당,서울특별시 마포구 상암동 상암월드컵파크11단지,마포구,상암동
...,...,...,...,...
3740,마곡엠밸리6단지경로당,서울특별시 강서구 마곡동,강서구,마곡동
3741,큰덕경로당,서울특별시 마포구 공덕동,마포구,공덕동
3742,구립양평3가경로당,서울특별시 영등포구 양평동3가 36번지,영등포구,양평동3가
3743,백련산힐스테이트1차아파트경로당,서울특별시 은평구 응암동,은평구,응암동


In [236]:
df_4_3_new = df_4_3.groupby('법정동').count()['자치구'].reset_index().rename(columns = {'자치구':'경로당 수'})
df_4_3_new

,법정동,경로당 수
0,가락동,24
1,가리봉동,4
2,가산동,10
3,가양동,29
4,가회동,1
...,...,...
321,효제동,2
322,효창동,4
323,후암동,5
324,휘경동,18


In [237]:
# df_4_3.to_csv('../data/raw_data/가중치선정/완료/4_3_경로당(주소).csv', index=False)
# df_4_3_new.to_csv('../data/raw_data/가중치선정/완료/4_3_경로당수.csv', index=False)

#### 4_4 보육시설 수

In [238]:
df_4_4 = pd.read_csv('../data/raw_data/가중치선정/보육시설+이용(동별)(2012년+이후)_20240411094627.csv')
df_4_4

,동별(1),동별(2),동별(3),2022,2022.1,2022.2,2022.3,2022.4,2022.5,2022.6,2022.7,2022.8,2022.9,2022.10,2022.11,2022.12,2022.13,2022.14,2022.15
0,동별(1),동별(2),동별(3),보육시설 정원,보육시설 정원,보육시설 정원,보육시설 정원,보육시설 정원,보육시설 정원,보육시설 정원,보육시설 정원,보육시설 현원,보육시설 현원,보육시설 현원,보육시설 현원,보육시설 현원,보육시설 현원,보육시설 현원,보육시설 현원
1,동별(1),동별(2),동별(3),소계,국공립,사회복지법인,법인·단체 등,민간,가정,부모협동,직장,소계,국공립,사회복지법인,법인·단체 등,민간,가정,부모협동,직장
2,합계,소계,소계,223628,103678,1533,4954,65662,25770,805,21226,167427,81969,991,3404,47079,21258,597,12129
3,합계,종로구,소계,4409,1993,100,49,193,39,-,2035,2642,1385,47,23,139,35,-,1013
4,합계,종로구,사직동,954,101,-,-,41,-,-,812,542,67,-,-,33,-,-,442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,합계,강동구,천호2동,819,428,-,-,331,-,-,60,622,331,-,-,262,-,-,29
450,합계,강동구,길동,1009,273,-,76,585,75,-,-,697,198,-,73,357,69,-,-
451,합계,강동구,강일동,1168,679,-,-,272,217,-,-,934,594,-,-,147,193,-,-
452,합계,강동구,상일1동,856,664,-,-,133,59,-,-,823,636,-,-,128,59,-,-


In [239]:
df_4_4 = df_4_4[['동별(2)', '동별(3)', '2022', '2022.8']].rename(columns = {'동별(2)': '자치구', '동별(3)':'행정동', '2022':'보육시설정원', '2022.8':'보육시설현원'}).iloc[3:]
df_4_4

,자치구,행정동,보육시설정원,보육시설현원
3,종로구,소계,4409,2642
4,종로구,사직동,954,542
5,종로구,삼청동,104,46
6,종로구,부암동,150,116
7,종로구,평창동,175,160
...,...,...,...,...
449,강동구,천호2동,819,622
450,강동구,길동,1009,697
451,강동구,강일동,1168,934
452,강동구,상일1동,856,823


In [240]:
df_4_4 = df_4_4[~df_4_4['행정동'].isin(['소계'])]
df_4_4

,자치구,행정동,보육시설정원,보육시설현원
4,종로구,사직동,954,542
5,종로구,삼청동,104,46
6,종로구,부암동,150,116
7,종로구,평창동,175,160
8,종로구,무악동,240,152
...,...,...,...,...
449,강동구,천호2동,819,622
450,강동구,길동,1009,697
451,강동구,강일동,1168,934
452,강동구,상일1동,856,823


In [241]:
df_4_4[df_4_4['보육시설정원'] == '-']

,자치구,행정동,보육시설정원,보육시설현원
370,서초구,반포본동,-,-
396,강남구,개포1동,-,-
429,송파구,잠실7동,-,-
446,강동구,둔촌1동,-,-


In [242]:
df_4_4 = df_4_4[~df_4_4['보육시설정원'].isin(['-'])].reset_index(drop = True)
df_4_4

,자치구,행정동,보육시설정원,보육시설현원
0,종로구,사직동,954,542
1,종로구,삼청동,104,46
2,종로구,부암동,150,116
3,종로구,평창동,175,160
4,종로구,무악동,240,152
...,...,...,...,...
417,강동구,천호2동,819,622
418,강동구,길동,1009,697
419,강동구,강일동,1168,934
420,강동구,상일1동,856,823


In [243]:
df_4_4['보육시설현원'] = df_4_4['보육시설현원'].astype('int')
df_4_4['보육시설정원'] = df_4_4['보육시설정원'].astype('int')
df_4_4['보육시설포화도'] = round(df_4_4['보육시설현원']/df_4_4['보육시설정원'], 2)
df_4_4

,자치구,행정동,보육시설정원,보육시설현원,보육시설포화도
0,종로구,사직동,954,542,0.57
1,종로구,삼청동,104,46,0.44
2,종로구,부암동,150,116,0.77
3,종로구,평창동,175,160,0.91
4,종로구,무악동,240,152,0.63
...,...,...,...,...,...
417,강동구,천호2동,819,622,0.76
418,강동구,길동,1009,697,0.69
419,강동구,강일동,1168,934,0.80
420,강동구,상일1동,856,823,0.96


In [244]:
# df_4_4.to_csv('../data/raw_data/가중치선정/완료/4_4_보육시설(어린이집)수.csv', index=False)

In [245]:
df_4_4.groupby(['자치구'])[['보육시설포화도']].mean().sort_values(by = '보육시설포화도').reset_index()

,자치구,보육시설포화도
0,종로구,0.624118
1,강남구,0.643810
2,중구,0.671333
3,금천구,0.680000
4,관악구,0.709048
5,성동구,0.710000
6,서초구,0.730000
7,도봉구,0.732143
8,은평구,0.734375
9,동대문구,0.736429
